## What has been done
- implemented instructions: ADD, SUB, MOV, ADDI, SUBI, LDI, LDA, STA, STP 
- format that it is expecting in txt file:
     - For r type instructions: Opcode, Rd, Rs, cin/offset (integer)
     - For i type instructions: Opcode, Rd, imm(in hex)
     - For l/s type instructions: Opcode, Rd, address(in hex)
## What needs to be done
- implement with mif library
- implement instructions: XSR, LCG, Jump instructions
- Ability to handle variable names

In [2]:
f = open("test.txt", "r")
# lines = [x.upper() for x in f.readlines()]

In [3]:
instructions = {"ADD": "r", "SUB": "r", "MOV": "r", "XSR": "r", "LCG": "r", "LDR": "r", "STR": "r", 
                "ADDI": "i", "SUBI": "i", "LDI": "i", 
                "STP": "s", 
                "LDA": "ls", "STA": "ls", 
                "JEQ": "j", "JLT": "j", "JAL": "j", "JR": "j", "JMP": "j"}
registers = {"R0": "000", "GP1": "001", "GP2": "010", "GP3": "011", "GP4": "100", 
             "GP5": "101", "SP": "110", "RA": "111"}
cin_r = {"C0" : "00", "C1": "01", "CMSB" : "10", "CC" : "11"}
lcg_r = {"MUL" : "00", "MAC": "01", "LOOP" : "10", "LL" : "11"}

In [4]:
#function to handle cin and s or offset
def checkcinrs(inst):
    opcode = inst[0]
    if opcode in ["ADD", "SUB", "MOV", "XSR"] :
        try:
            cin = cin_r[inst[3]]
        except:
            cin = "00"
        try:
            s = '{0:01b}'.format(int(inst[4]))
        except:
            s = "0"
        return (cin + s)
    
    elif opcode == "LCG":
        try:
            lcg_mode = lcg_r[inst[1]]
        except:
            lcg_mode = "00"
        try:
            s = '{0:01b}'.format(int(inst[2]))
        except:
            s = "0"
        return (lcg_mode + s)
    else:
        try:
            return ('{0:03b}'.format(int(inst[3])))
        except:
            return "000"

#handle r type instructions
def check_r(inst):
    opcode = inst[0]
    if opcode == "LCG":
        cins = checkcinrs(inst)
        bin_inst = "0000100" + cins + "000000"
        print(bin_inst)
        return(int(bin_inst,2))
    
    assert(inst[1] in registers.keys() and inst[2] in registers.keys())
    Rd = registers[inst[1]]
    Rs = registers[inst[2]]

    
            
    if opcode == "ADD":
        cins = checkcinrs(inst)
        bin_inst = "0000000" + cins + Rs + Rd
        return(int(bin_inst,2))
        
    if opcode == "SUB":
        cins = checkcinrs(inst)
        # print(cins)
        bin_inst = "0000001" + cins + Rs + Rd
        # print(bin_inst)
        return(int(bin_inst,2))
        
    if opcode == "MOV":
        cins = checkcinrs(inst)
        bin_inst = "0000010" + cins + Rs + Rd
        # print(bin_inst)
        return(int(bin_inst,2))
        
    if opcode == "XSR":
        cins = checkcinrs(inst)
        bin_inst = "0000011" + cins + Rs + Rd
        # print(bin_inst)
        return(int(bin_inst,2))
        
    if opcode == "LDR":
        offset = checkcinrs(inst)
        bin_inst = "0000101" + offset + Rs + Rd
        # print(bin_inst)
        return(int(bin_inst,2))
        
    if opcode == "STR":
        offset = checkcinrs(inst)
        bin_inst = "0000110" + offset + Rs + Rd
        # print(bin_inst)
        return(int(bin_inst,2))

    
    

In [48]:

# Function to find two's complement of the immediate part of the i type instructions
def findTwoscomplement(str): 
    str = list(str) 
    n = len(str) 
  
    # Traverse the string to get first '1' from the last of string 
    pastfirstone = False
    for i, x in reversed(list(enumerate(str))):
        if (x == '1' and pastfirstone == False): 
            pastfirstone = True
            pass
        elif pastfirstone == True:
            if (str[i] == '1'): 
                str[i] = '0'
            else: 
                str[i] = '1'
  
    str = ''.join(str) 
    return str

#handle r type instructions
def check_i(inst):
    assert(inst[1] in registers.keys())
    Rd = registers[inst[1]]
    opcode = inst[0]
    try:
        imm_int = int(inst[2],16)
#         print(imm_int)
        if imm_int < 0:
            flipped = '{0:012b}'.format(-imm_int)
            imm = findTwoscomplement(flipped)
        else:
            imm = '{0:012b}'.format(imm_int)
        # print(imm)
    except:
        imm = "000000000000"
        
    if opcode == "ADDI":
        bin_inst = "000100" + imm[-7:] + Rd
#         print(bin_inst)
        return(int(bin_inst,2))
        
    if opcode == "SUBI":
        bin_inst = "000101" + imm[-7:] + Rd
#         print(bin_inst)
        return(int(bin_inst,2))
        
    if opcode == "LDI":
        bin_inst = "000110" + imm[-7:] + Rd
        # print(bin_inst)
        return(int(bin_inst,2))

In [49]:

#handle ls type instructions
def check_ls(inst):
    #check if the register being accessed are one of the first 4 registers
    assert(inst[1] in list(registers.keys())[:4])
    Rd = registers[inst[1]][1:]
    opcode = inst[0]
    try:
        a = '{0:011b}'.format(int(inst[2],16))
    except:
        a = "00000000000"
    if opcode == "LDA":
        bin_inst = "010" + a + Rd
        # print(bin_inst)
        return(int(bin_inst,2))
        
    if opcode == "STA":
        bin_inst = "011" + a + Rd
        # print(bin_inst)
        return(int(bin_inst,2))

In [50]:
#check address
def findaddress(address):
    if address.isnumeric() == True:
        a = '{0:011b}'.format(int(address,16))
    elif address in variables.keys():
        a = '{0:011b}'.format(int(variables[address],16))
    else:
        a = "00000000000"
    return a

#handle j type instructions
def check_j(inst):
    opcode = inst[0]
    #JMP instruction will have to directly compare with R0 which is all zeros 
    if opcode == "JMP":
        a = findaddress(inst[1])
        bin_inst = "100" + a + "00"
#         print(bin_inst)
        return(int(bin_inst,2))
    
    # JAL instructions do not need a register
    if opcode == "JAL":
        a = findaddress(inst[1])
        bin_inst = "110" + a + "00"
#         print(bin_inst)
        return(int(bin_inst,2))
    
    #JR ignores other parameters in instruction as it directly deals with RA
    if opcode == "JR":
        bin_inst = "111" + '0000000000000'
#         print(bin_inst)
        return(int(bin_inst,2))

    #check if the register being accessed are one of the first 4 registers
    assert(inst[1] in list(registers.keys())[:4])
    Rd = registers[inst[1]][1:]
    a = findaddress(inst[2])
    #JEQ and JLT instructions access the first 4 registers (R0 to GP)
    if opcode == "JEQ":
        bin_inst = "100" + a + Rd
#         print(bin_inst)
        return(int(bin_inst,2))

    if opcode == "JLT":
        bin_inst = "101" + a + Rd
#         print(bin_inst)
        return(int(bin_inst,2))

In [51]:
#convert string intructions to hexadecimal values
def convert_to_hex(lines):
    assembly = []
    #Loop through each line of instruction and do checks based on opcode type
    for line in lines:
        inst = line.strip('\n').split(" ")
        print(inst)
        assert(inst[0] in instructions.keys())
        opcode = inst[0]
        #r type instruction check
        if instructions[opcode] == "r":
            assembly.append(check_r(inst))
        #i type instruction check
        elif instructions[opcode] == "i":
            assembly.append(check_i(inst))
        #s type instruction check
        elif instructions[opcode] == "s":
            bin_inst = "000111" + "0000000000"
            assembly.append(int(bin_inst,2))
        #ls type instruction check
        elif instructions[opcode] == "ls":
            assembly.append(check_ls(inst))
        #j type instruction check
        else: 
            assembly.append(check_j(inst))
    return assembly



In [68]:
#open text file and input into a list of string instructions
f = open("test.txt", "r")
"""Starts with an alphabetic character or _. Followed by zero or more alpha-numeric characters or _."""
import re
variables = {}
lines = []
data = {}
data_pos = False
data_starting_pos = 0
count = 0 
for idx, line in enumerate(f.readlines()):
    if data_pos == True:
        data_value = re.sub(r"^\s+|\s+$", "", line)
        data[count + data_starting_pos] = int(data_value.split(' ')[1],16)
        count += 1
    else:
        if line[:3] == "ORG":
            data_pos = True
            data_starting_pos = int(line.split(' ')[1],16)
        else:
            if ":" in line:
                variable = line.split(':')[0]
                assert((variable[0].isalpha() or variable[0]=="_") and (variable[1:].isalnum())or variable[0]=="_")
                variables[variable] = '{0:x}'.format(idx)
                instruction = line.split(':')[1]
                #remove leading and trailing spaces
                lines.append(re.sub(r"^\s+|\s+$", "", instruction))
            else:
                lines.append(re.sub(r"^\s+|\s+$", "", line))


#convert string intructions to hexadecimal values
test = (convert_to_hex(lines))
print(variables)
print(data)

['LDI', 'GP1', '5']
['LDI', 'SP', '0x0FFC']
['JLT', 'GP1', 'returnOne']
['SUBI', 'SP', '0x03']
['STR', 'RA', 'SP', '0']
['STR', 'GP1', 'SP', '1']
['SUBI', 'GP1', '1']
['JAL', 'fib']
['STR', 'GP2', 'SP', '2']
['LDR', 'GP1', 'SP', '1']
['SUBI', 'GP1', '2']
['JAL', 'fib']
['LDR', 'GP3', 'SP', '2']
['ADD', 'GP2', 'GP3']
['LDR', 'RA', 'SP', '0']
['ADDI', 'SP', '0x03']
['JR', 'RA']
['LDI', 'GP2', '1']
['JR', 'RA']
{'fib': '2', 'returnOne': '11'}
{}


In [69]:
['{0:4x}'.format(x) for x in test]

['1829',
 '1be6',
 'a045',
 '141e',
 ' c37',
 ' c71',
 '1409',
 'c008',
 ' cb2',
 ' a71',
 '1411',
 'c008',
 ' ab3',
 '  1a',
 ' a37',
 '101e',
 'e000',
 '180a',
 'e000']

In [70]:
list(data.keys())

[]

In [72]:
#write hexadecimal into mif file directly
def write_to_mif(f_out, hex_inst, width=16, depth=4096):
    assert(len(hex_inst) <= depth)
    buf = 'WIDTH={:d};\nDEPTH={:d};\nADDRESS_RADIX=HEX;\nDATA_RADIX=HEX;\nCONTENT BEGIN\n'.format(width, depth)
    l_index = str(len('{:x}'.format(depth)))
    l_data = str(int(width / 4))
    s = '\t{:0' + l_index + 'X} : {:0' + l_data + 'X};\n'
    for i, j in enumerate(hex_inst):
        buf += s.format(i, j)
    if data:
        origin = min(list(data.keys()))
        if len(buf) < origin:
            buf += ('\t[{:0' + l_index + 'X}..{:0' + l_index + 'X}] : {:0' + l_data + 'X};\n').format(len(hex_inst), origin - 1, 0)
        for i in list(data.keys()):
            buf += s.format(i, data[i])
        buf += ('\t[{:0' + l_index + 'X}..{:0' + l_index + 'X}] : {:0' + l_data + 'X};\n').format(max(list(data.keys())) + 1, depth - 1, 0)
        
    else:
        if len(buf) == depth - 1:
            buf += s.format(depth - 1, 0)
        elif len(buf) < depth - 1:
            buf += ('\t[{:0' + l_index + 'X}..{:0' + l_index + 'X}] : {:0' + l_data + 'X};\n').format(len(hex_inst), depth - 1, 0)
    buf += 'END;\n'
    print(buf)
#     with open(f_out, 'w') as f:
#         f.write(buf)

write_to_mif("test.mif",test)

WIDTH=16;
DEPTH=4096;
ADDRESS_RADIX=HEX;
DATA_RADIX=HEX;
CONTENT BEGIN
	0000 : 1829;
	0001 : 1BE6;
	0002 : A045;
	0003 : 141E;
	0004 : 0C37;
	0005 : 0C71;
	0006 : 1409;
	0007 : C008;
	0008 : 0CB2;
	0009 : 0A71;
	000A : 1411;
	000B : C008;
	000C : 0AB3;
	000D : 001A;
	000E : 0A37;
	000F : 101E;
	0010 : E000;
	0011 : 180A;
	0012 : E000;
	[0013..0FFF] : 0000;
END;



In [64]:
if data:
   print(data) 

{2048: 512, 2049: 2048}
